# 6. Osztályok, filekezelés
_2021.08.29_

Az órán ...

Tartalom:
- Osztályok
- Fájlkezelés (alapértelmezett)
- Hibakezelés

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## 6.1 Osztályok, öröklődés
### 6.1.1 Scope, namespaces

In [2]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


### 6.1.2 Classes

In [3]:
class MyClass:
    my_var = 12345

    def my_func(self):
        return 'test'

In [4]:
x = MyClass()

In [5]:
x.my_var

12345

In [6]:
x.my_func()

'test'

### 6.1.3 Instatiation

In [7]:
class MyClass:
    my_var = 12345
    def __init__(self):
        print('__init__ function called')
        self.my_var = 54321

    def my_func(self):
        return 'test'

In [8]:
x = MyClass()

__init__ function called


In [9]:
x.my_var

54321

### 6.1.4 Class & instance variables

In [10]:
class Kutya:
    fajta = 'kutya' # osztályváltozó

    def __init__(self, nev):
        self.nev = nev    # példányváltozó

In [11]:
d = Kutya('Buksi')
e = Kutya('Blöki')

In [12]:
d.fajta, d.nev

('kutya', 'Buksi')

In [13]:
e.fajta, e.nev

('kutya', 'Blöki')

> **1. feladat**: Egészítsd ki a Kutya osztályt egy tanul_trukk függvénnyel, amely a paraméterként átadott trükköt hozzáadja az adott kutya megtanult trükkjei közé!

Rossz megoldás

In [14]:
class Kutya:
    fajta = 'kutya' # osztályváltozó
    trukkok = []

    def __init__(self, nev):
        self.nev = nev    # példányváltozó

    def tanul_trukk(self, trukk):
        self.trukkok.append(trukk)

In [15]:
d = Kutya('Fido')
e = Kutya('Mancika')
d.tanul_trukk('ül')
d.tanul_trukk('forog')

In [16]:
d.trukkok

['ül', 'forog']

In [17]:
e.trukkok

['ül', 'forog']

Jó megoldás

In [18]:
class Kutya:
    fajta = 'kutya' # osztályváltozó

    def __init__(self, nev):
        self.nev = nev    # példányváltozó
        self.trukkok = []

    def tanul_trukk(self, trukk):
        self.trukkok.append(trukk)

In [19]:
d = Kutya('Fido')
e = Kutya('Mancika')
d.tanul_trukk('ül')
d.tanul_trukk('forog')

In [20]:
d.trukkok

['ül', 'forog']

In [21]:
e.trukkok

[]

### 6.1.5 Inheritance

In [22]:
class Jarmu:
    def __init__(self, szin):
        self.szin = szin
    
    def megjelenit(self):
        print('A jármű színe: {:}'.format(self.szin))

In [23]:
j = Jarmu('piros')
j.megjelenit()

A jármű színe: piros


In [24]:
class Auto(Jarmu):
    def __init__(self, szin, kerekek_szama):
        Jarmu.__init__(self, szin)
        #super().__init__(szin)
        
        self.kerekek_szama = kerekek_szama

    def megjelenit(self):
        print('A jármű színe: {:}\nKerekeinek száma: {:}'.format(self.szin, self.kerekek_szama))

class Repulo(Jarmu):
    pass

In [25]:
a = Auto('kek', 4)

In [26]:
a.megjelenit()

A jármű színe: kek
Kerekeinek száma: 4


> **2. feladat**: Fejezd be a Repulo osztályt, paraméterként adj hozzá egy Boolean típusú változót, amely azt tárolja, hogy a repülőgép futóműve behúzható-e!

In [27]:
class Repulo(Jarmu):
    def __init__(self, szin, behuzhato):
        super().__init__(szin)
        
        self.behuzhato = behuzhato

    def megjelenit(self):
        print('A jármű színe: {:}\nFutómű behúzható: {:}'.format(self.szin, self.behuzhato))

In [28]:
r = Repulo('zöld', False)
r.megjelenit()

A jármű színe: zöld
Futómű behúzható: False


## 6.2 Filekezelés
Olvassuk be a 6.2_airtravel.csv file tartalmát, amely a havi transzatlanti polgári repülések számát tartalmazza. Az egyes számok ezer utast jelentenek. A file-ok megnyitására az **open(filenév, mód)** parancs használható, ahol a
- _filenév_: a megnyitni kívánt file neve
- _mód_:
    - r: reading, megnyitás olvasásra (alapértelmezett)
    - w: writing: megnyitás írásra
    - a: append: megnyitás hozzáfűzésre
    - r+: megnyitás olvasásra és írásra

In [34]:
file = open('6.2_airtravel.csv', 'r')

In [32]:
print (file.read())

"Month", "1958", "1959", "1960"
"JAN",  340,  360,  417
"FEB",  318,  342,  391
"MAR",  362,  406,  419
"APR",  348,  396,  461
"MAY",  363,  420,  472
"JUN",  435,  472,  535
"JUL",  491,  548,  622
"AUG",  505,  559,  606
"SEP",  404,  463,  508
"OCT",  359,  407,  461
"NOV",  310,  362,  390
"DEC",  337,  405,  432




In [35]:
for each in file:
    print (each)

"Month", "1958", "1959", "1960"

"JAN",  340,  360,  417

"FEB",  318,  342,  391

"MAR",  362,  406,  419

"APR",  348,  396,  461

"MAY",  363,  420,  472

"JUN",  435,  472,  535

"JUL",  491,  548,  622

"AUG",  505,  559,  606

"SEP",  404,  463,  508

"OCT",  359,  407,  461

"NOV",  310,  362,  390

"DEC",  337,  405,  432





In [55]:
file = open('test.txt', 'w', encoding='utf-8')
file.write("Szöveg írása fileba")
file.close()

In [56]:
file = open('test.txt', 'a', encoding='utf-8')
file.write("\nMég egy sor")
file.close()

A with parancs használatával nem szükséges lezárni a file-t, mivel az automatikusan lezáródik.

In [59]:
with open("test.txt", encoding = 'utf-8') as file:  
    data = file.read()

In [60]:
data

'Szöveg írása fileba\nMég egy sor'

In [68]:
f = open("test.txt", encoding = 'utf-8')

In [69]:
f.read(6)

'Szöveg'

In [70]:
f.read(7)

' írása '

In [71]:
f.tell()

16

In [72]:
f.read()

'fileba\nMég egy sor'

In [73]:
f.tell()

36

In [74]:
f.seek(0)

0

In [75]:
f.read(13)

'Szöveg írása '

In [76]:
f.seek(0)

0

In [77]:
f.readline()

'Szöveg írása fileba\n'

In [78]:
f.readline()

'Még egy sor'

In [79]:
f.seek(0)

0

In [80]:
f.readlines()

['Szöveg írása fileba\n', 'Még egy sor']

## 6.3 Hibakezelés

In [81]:
10/0

ZeroDivisionError: division by zero

In [82]:
try:
    print(10/0)
except ZeroDivisionError:
    print("Véletlenül 0-val osztottunk")

Véletlenül 0-val osztottunk


In [83]:
raise NameError('Hello')

NameError: Hello

In [93]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("Nullával nem lehet osztani!")
    else:
        print("Az eredmény: ", result)
    finally:
        print("Finally")

In [87]:
divide(2, 1)

Az eredmény:  2.0
Finally


In [88]:
divide(2, 0)

Nullával nem lehet osztani!
Finally


In [94]:
divide("2", "1")

Finally


TypeError: unsupported operand type(s) for /: 'str' and 'str'

> **1. feladat**: Egészítsd ki a divide függvényt, hogy helytelen típusú input esetén is dobjon kivételt!

In [96]:
def divide(x, y):
    try:
        result = x / y
    except TypeError:
        print("Az input paraméterek típusa vagy float vagy integer kell legyen.")
    except ZeroDivisionError:
        print("Nullával nem lehet osztani!")
    else:
        print("Az eredmény: ", result)
    finally:
        print("Finally")